In [ ]:
!pip install impyute

In [ ]:
from impyute.imputation.cs import fast_knn
import numpy as np
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.notebook import tqdm
from collections import defaultdict

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')
sub = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

In [ ]:
F4_col_list = []
for col in data.columns:
    if "F_4" in col:
        F4_col_list.append(col)

In [ ]:
def nan_indices(data):
    return np.argwhere(np.isnan(data))

In [ ]:
nan_xy = nan_indices(data[F4_col_list].values)

In [ ]:
# impute from other sub
other_sub = pd.read_csv("../input/tps2206-gbm-resnet-imputation/submission.csv", index_col='row-col')
other_sub_dic = defaultdict(list)
for i in tqdm(other_sub.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    val = other_sub.loc[i, 'value']
    other_sub_dic[col].append((row, val))

for col in tqdm(data.columns):
    if not other_sub_dic[col]:
        continue
    index_value_list = other_sub_dic[col]
    index_list = []
    value_list = []
    for index, value in index_value_list:
        index_list.append(index)
        value_list.append(value)
    data[col].loc[index_list] = value_list

In [ ]:
data_c = data[F4_col_list].values

I try add tqdm from this code
https://impyute.readthedocs.io/en/master/user_guide/overview.html

In [ ]:
kdtree = KDTree(data_c, leafsize=1000)

In [ ]:
def to_percentage(vec):
    """ Converts list of real numbers into a list of percentages """
    return vec/np.sum(vec)

def shepards(distances, power=2):
    return to_percentage(1/np.power(distances, power))

In [ ]:
# fast knn with k-d tree
k=100
for i, (x_i, y_i) in enumerate(tqdm(nan_xy)):
    distances, indices = kdtree.query(data_c[x_i], k=k+1, eps=1e-5, p=2)
    distances = distances[1:]
    indices = indices[1:]
    distances += 1e-10
    weights = shepards(distances)
    data_c[x_i][y_i] = np.dot(weights, [data_c[ind][y_i] for ind in indices])
    if (i+1) % 10_000 == 0:
        print(i+1, distances.mean(), distances.std())

In [ ]:
data[F4_col_list] = data_c

In [ ]:
ind_list = []
val_list = []
for i in tqdm(sub.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    val = data[col][row]
    ind_list.append(i)
    val_list.append(val)

sub['value'].loc[ind_list] = val_list

In [ ]:
sub.to_csv("submission.csv", index=True)
sub